In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib notebook
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
#  Need plotly > 4.8 to do plotting px plotting well

In [2]:
def contiguous_regions(condition):
    # https://stackoverflow.com/questions/4494404/find-large-number-of-consecutive-values-fulfilling-condition-in-a-numpy-array    idx = []
    idx = []
    i = 0
    while i < len(condition):
        x1 = i + condition[i:].argmax()
        try:
            x2 = x1 + condition[x1:].argmin()
        except Exception as e:
            print(e)
            x2 = x1 + 1
        if x1 == x2:
            if condition[x1] == True:
                x2 = len(condition)
            else:
                break
        idx.append( [x1,x2] )
        i = x2
    return idx

In [3]:
a= np.arange(10)

b = a>5
#b[9]=0
b[7:].argmin()
contiguous_regions(b)
b[6:10]

array([ True,  True,  True,  True])

In [4]:
def plot_df(df):
    labels = ['mean37', 'n37', 'var37']
    labels = ['mean38', 'n38', 'var38']
    labels = ['mean39', 'n39', 'var39']
    
    fig = go.FigureWidget()
    def add(fig, labels, color):
        var = (df[labels[2]]/df[labels[1]])**0.5
        # print(var)
        fig.add_scatter(x = df['time'], y=-df[labels[0]], 
                      error_y = dict(type='data', array=var, visible=True),
                      mode='markers', name=labels[1], 
                      marker = dict(size=5, symbol='circle', color=color)
                       )

                     
    labels = ['mean39', 'n39', 'var39']
    add(fig, labels, 'green')
    labels = ['mean37', 'n37', 'var37']
    add(fig, labels, 'red')
    labels = ['mean38', 'n38', 'var38']
    add(fig, labels, 'blue')
    
    return fig

In [5]:
files = glob.glob('data/water/*20200615*.bin.csv')
files.sort()
files = files[::-1]
for file in files:
    print(file.split('_')[1])
    

NIST0005
NIST0004
NIST0003
NIST0001
NIST0000
NIST-GEN


In [7]:
df = pd.read_csv(files[0])
# df.drop((df[df[' epoch_time']<3]).index, inplace=True)
df['# time'] = pd.to_datetime(df['# time'])
df.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
    # cleanup whitespaces in data
df = df.applymap(lambda x: x.strip() if type(x)==str else x)

        
# t3 = pd.to_datetime('2020-06-14 18:00:00')
# df.drop((df[df['time']<t3]).index, inplace=True)



In [ ]:
hist_id = np.array(np.unique(df['encounter_id'],return_counts=True))
good_id = hist_id[0, hist_id[1,:]>1] 

df_by_id = {}
for id in good_id:
    df_by_id[id] = df[df['encounter_id']==id]
    #print(df_by_id[id])

In [ ]:
for i in range(len(good_id)):
    # print(good_id[i])
    fig = plot_df(df_by_id[good_id[i]])
    fig.update_layout(
        title=good_id[i][:8],
        xaxis_title="time",
        yaxis_title="rssi"
    )
    fig.show()

In [32]:
def load_et_data(file):
    # load csv file into pandas dataframe, cleanup names, rssi, etc
    df = pd.read_csv(file)
    # df.drop((df[df[' epoch_time']<3]).index, inplace=True)
    df['# time'] = pd.to_datetime(df['# time'])
    df.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
    # cleanup whitespaces in data
    df = df.applymap(lambda x: x.strip() if type(x)==str else x)
    # Make rssi negative... 
    for k in  df.keys():
        if k.startswith('mean'):
            df[k]=-df[k]
    #  Clean up time        
    t3 = pd.to_datetime('2020-06-14 20:10:00')
    df.drop((df[df['time']<t3]).index, inplace=True)
    df['rssi_max']=df.loc[:,['mean37','mean38','mean39']].max(axis=1)
    
    return df

In [42]:
for file in files[:-1]:
    #print(file)
    df = load_et_data(file)
            
    fig = px.scatter(df, x="time",
#                      y=["mean37","mean38","mean39"],
                     y=["rssi_max"],
                     color="encounter_id",
                     facet_col="variable")
    for trace in fig.data:
        trace.name = trace.name[:4]
    fig.update_layout(title = {
                       'text': file.split('_')[1],
                       'xanchor': 'left',
                       'yanchor': 'top'})
    fig.show()

In [43]:
hist_id = np.array(np.unique(df['encounter_id'],return_counts=True))
good_id = hist_id[0, hist_id[1,:]>1] 

df_by_id = {}
for id in good_id:
    df_by_id[id] = df[df['encounter_id']==id]
    #print(df_by_id[id])

In [63]:
for k in df_by_id.keys():
    regions = contiguous_regions(df_by_id[k]['rssi_max']>-60)
    #regions = np.array(regions)
    #cut = regions[:,1]-regions[:,0] > 6
    #print(k[:4], df_by_id[k]["time"][regions[cut].flatten()])

KeyError: 361

In [70]:
for k in df_by_id.keys():
    print(df_by_id[k])
    #print(np.diff(df_by_id[k]['time']))

                    time  epoch_time  first_sec  last_sec  mean37  n37  min37  \
5    2020-06-14 20:20:00    26536460          0        58     -76    9     75   
8    2020-06-14 20:17:00    26536457          1        56     -77    8     71   
12   2020-06-14 20:18:00    26536458          0        58     -76    5     78   
15   2020-06-14 20:19:00    26536459          0        58     -76    7     74   
21   2020-06-14 20:20:00    26536460          0        58     -76    9     75   
...                  ...         ...        ...       ...     ...  ...    ...   
1718 2020-06-14 22:31:00    26536591          3        58     -73    6     72   
1719 2020-06-14 22:32:00    26536592          0        58     -73    9     72   
1727 2020-06-14 22:33:00    26536593          1        51     -73    5     73   
1732 2020-06-14 22:34:00    26536594          3        58     -72    5     72   
1738 2020-06-14 22:35:00    26536595          3        58     -72    7     72   

      max37  var37  mean38 

In [69]:
df


,time,epoch_time,first_sec,last_sec,mean37,n37,min37,max37,var37,mean38,...,var38,mean39,n39,min39,max39,var39,crypto_flag,extra_flag,encounter_id,rssi_max
0,2020-06-14 20:19:00,26536459,0,58,-54,8,54,56,32,-62,...,42,-63,9,57,75,408,7,0,6681DC25DD56D6211911318B1BFFBE47589D08745E82D9...,-54
1,2020-06-14 20:19:00,26536459,1,58,-57,7,56,57,17,-59,...,14,-62,9,61,63,3,7,0,4B3CCE1AE7426014CDA00E27BB98A91873D5D5A31B33BE...,-57
2,2020-06-14 20:19:00,26536459,3,58,-52,7,56,61,192,-54,...,44,-60,6,44,67,351,7,0,508F9FBD6885447454AA9E18263F018A3B0ED566DA91BE...,-52
3,2020-06-14 20:20:00,26536460,0,51,-58,7,57,59,5,-60,...,0,-56,5,56,56,0,7,0,508F9FBD6885447454AA9E18263F018A3B0ED566DA91BE...,-56
4,2020-06-14 20:20:00,26536460,0,58,-57,8,56,57,2,-59,...,2,-61,6,61,61,0,7,0,A71DE62A10F0BC5BD2562D2C532F78E0D274A3FA095A8E...,-57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2020-06-14 22:35:00,26536595,0,58,-64,8,64,64,0,-65,...,4,-62,8,62,62,0,7,0,6681DC25DD56D6211911318B1BFFBE47589D08745E82D9...,-62
1735,2020-06-14 22:35:00,26536595,0,58,-57,9,56,57,1,-60,...,0,-64,7,63,64,0,7,0,4B3CCE1AE7426014CDA00E27BB98A91873D5D5A31B33BE...,-57
1736,2020-06-14 22:35:00,26536595,1,58,-68,9,68,70,13,-63,...,2,-67,7,66,67,2,7,0,A71DE62A10F0BC5BD2562D2C532F78E0D274A3FA095A8E...,-63
1737,2020-06-14 22:35:00,26536595,1,56,-57,6,57,57,0,-54,...,2,-63,7,63,63,0,7,0,508F9FBD6885447454AA9E18263F018A3B0ED566DA91BE...,-54
